In [2]:
from parser import *
from snapshoter import *
from language_extractor import *
from plotter import *
import pandas as pd
import os
from tabulate import tabulate
import altair as alt

## Loading the dataset

In [3]:
df = pd.read_csv('../dataset/200_workflowsonly.csv')
print(f"Total number of records: {df.shape[0]}")

# Drop the records that were deleted from the dataset
df = df.dropna(subset=['file_hash'])

# Force the type of the valid_yaml column to be boolean
# (After a check it appears that the column is boolean but better safe than sorry)
df['valid_yaml'] = df['valid_yaml'].astype(bool)

Total number of records: 2595399


## Counting the number of invalid yaml files per year

In [ ]:
# Total number of invalid records in the dataset
nbr_invalid_yaml = df[df['valid_yaml'] == False].shape[0]
print(f"Number of invalid records: {nbr_invalid_yaml}")

df['committed_date'] = pd.to_datetime(df['committed_date'], unit='s')
df['committed_year'] = df['committed_date'].dt.year

# Only keep the records that are invalid and then sort them by year
invalid_df = df[df['valid_yaml'] == False]
invalids_per_year = invalid_df['committed_year'].value_counts().sort_index()

print("Number of invalid records per year: ")
print(invalids_per_year)

## Extracting information from a single workflow file


In [3]:
firstWorkflow = df.iloc[0]
file_hash = firstWorkflow['file_hash']

folder_path = f"../dataset/workflows"
file_path = os.path.join(folder_path, file_hash)

In [4]:
if not os.path.isfile(file_path):
    print(f"File {file_path} not found.")
else:
    parsed_data = parse_workflow(file_path)

    if parsed_data:
        print(f"Workflow analysé : {parsed_data['file_path']}")
        print(f"- Nombre de lignes : {parsed_data['lines_count']}")
        print(f"- Déclencheurs : {parsed_data['events']}")
        print(f"- Nombre de jobs : {parsed_data['jobs_count']}")

        for job, details in parsed_data["jobs"].items():
            print(f"  * Job: {job}")
            print(f"    - Nombre de steps: {details['steps_count']}")
            print(f"    - Utilise GitHub Actions: {details['uses_github_actions']}")
            print(f"    - Utilise des commandes shell: {details['uses_commands']}")
            print(f"    - Détails des steps :")
            for step in details["step_details"]:
                print(f"      - {step['name']}: uses={step['uses']}, run={step['run']}")

Workflow analysé : ../dataset/workflows\6f1b3e145aeb96e714212eb0b5e05088d7d6b50a4a945a3488779a94a85c80f0
- Nombre de lignes : 27
- Déclencheurs : ['push', 'pull_request']
- Nombre de jobs : 1
  * Job: build
    - Nombre de steps: 6
    - Utilise GitHub Actions: True
    - Utilise des commandes shell: True
    - Détails des steps :
      - Checkout code: uses=actions/checkout@v3, run=None
      - Setup node: uses=actions/setup-node@v3, run=None
      - Install dependencies: uses=None, run=if ! yarn install; then
  cat /tmp/xfs-*/buildfile.log 2>/dev/null || true
  exit 1
fi

      - Lint: uses=None, run=yarn lint
      - Check types: uses=None, run=yarn types:check
      - Build: uses=None, run=yarn build:chrome


## Snapshot without filtering

In [3]:
unfiltered_snapshots = []
year = 2019
while year <= 2024:
    snapshot = extract_snapshot(df,year)
    unfiltered_snapshots.append((year,snapshot))
    print(f"Year : {year}")
    print(f"Number of workflows in the snapshot : {snapshot.shape[0]}\n")
    year += 1

Year : 2019
Number of workflows in the snapshot : 1905

Year : 2020
Number of workflows in the snapshot : 31145

Year : 2021
Number of workflows in the snapshot : 79553

Year : 2022
Number of workflows in the snapshot : 127814

Year : 2023
Number of workflows in the snapshot : 179073

Year : 2024
Number of workflows in the snapshot : 219401



## Snapshot with filtering the invalid workflows

- Here we only consider the workflows that are valid (valid_yaml = True).
- The other workflows will be dropped from the resulting snapshot dataframe


In [4]:
# This block of code is only used to gather information
# on the number of invalid records

invalid_yaml = df.loc[df['valid_yaml'] == False]
print(f"Number of invalid records : {len(invalid_yaml)}\n")

Number of invalid records : 15417



In [5]:
filtered_snapshots = []

filtered_df = delete_invalid_yaml_records(df)

year = 2019
while year <= 2024:
    snapshot_filtered = extract_snapshot(filtered_df, year)
    filtered_snapshots.append((year,snapshot_filtered))
    print(f"Year: {year}")
    print(f"Number of workflows in the snapshot: {snapshot_filtered.shape[0]}\n")
    year += 1


Year: 2019
Number of workflows in the snapshot: 1902

Year: 2020
Number of workflows in the snapshot: 31092

Year: 2021
Number of workflows in the snapshot: 79476

Year: 2022
Number of workflows in the snapshot: 127675

Year: 2023
Number of workflows in the snapshot: 178886

Year: 2024
Number of workflows in the snapshot: 219186



## Get language repartition

In [6]:
language_repartitions = []

for year, snapshot in filtered_snapshots:
    language_repartitions.append((year, len(snapshot), extract_languages_by_repository(snapshot)))

print("Languages counts: ")
for year, repositories, language_count in language_repartitions:
    print(f"Year {year}:")
    print(language_count)
    print()


Languages counts: 
Year 2019:
{'TypeScript': 231, 'Go': 139, 'JavaScript': 128, 'Python': 126, 'Java': 96, 'Rust': 93, 'C++': 65, 'Ruby': 62, 'PHP': 55, 'C': 46, 'C#': 43, 'Kotlin': 28, 'Shell': 27, 'Swift': 22, 'Elixir': 12, 'Scala': 9, 'Haskell': 7, 'Jupyter Notebook': 6, 'Lua': 6, 'Dart': 6, 'Dockerfile': 5, 'Vue': 5, 'Perl': 4, 'Objective-C': 3, 'Common Lisp': 3, 'Julia': 3, 'Clojure': 2, 'Erlang': 2, 'Fortran': 2, 'Groovy': 2, 'R': 2, 'TeX': 2, 'Racket': 1, 'Nix': 1, 'OCaml': 1, 'F#': 1, 'Zig': 1, 'Elm': 1}

Year 2020:
{'Python': 1998, 'TypeScript': 1666, 'Go': 1147, 'JavaScript': 1134, 'Java': 987, 'C++': 892, 'Rust': 878, 'PHP': 610, 'C#': 535, 'C': 523, 'Ruby': 370, 'Julia': 314, 'Kotlin': 291, 'Shell': 286, 'R': 221, 'Scala': 152, 'Swift': 145, 'Dart': 104, 'Dockerfile': 97, 'Jupyter Notebook': 97, 'Haskell': 93, 'Elixir': 74, 'Vue': 61, 'Lua': 47, 'OCaml': 38, 'Clojure': 33, 'Groovy': 28, 'Perl': 26, 'F#': 24, 'Nix': 23, 'Objective-C': 21, 'Makefile': 20, 'TeX': 20, 'Erlang':

## Extract percentage of languages for each year

- Make a table for each year with the number of repositories using the corresponding language and the percentage
- If the percentage of a row is less them 1% then group them as Other

In [7]:
resulting_repartition = []

for year, nbr_repositories, repartition in language_repartitions:
    # Create a dataframe with the languages and their counts
    languages_df = pd.DataFrame(repartition.items(), columns=['Language', 'Count'])
    # Compute the percentage for each language
    languages_df['Percentage'] = (languages_df['Count'] / nbr_repositories) * 100
    # Sort the dataframe by count
    languages_df = languages_df.sort_values(by='Count', ascending=False)
    languages_df['Count'] = languages_df['Count'].astype(int)

    '''
    Now we need to group the languages with less than 1% of total occurrences, group tem as Other and delete the rows.
    '''
    # Create a new Dataframe with the grouped languages
    less_represented = languages_df[languages_df['Percentage'] < 1].sum(numeric_only=True)
    less_represented['Language'] = 'Other'

    # Deleting the rows with less than 1% in the original df by sorting it before concatenating with the above df
    languages_df = languages_df[languages_df['Percentage'] >= 1]
    languages_df = pd.concat([languages_df, pd.DataFrame([less_represented])], ignore_index=True)

    resulting_repartition.append((year,languages_df))

    table = tabulate(languages_df, headers='keys', tablefmt='psql')

    print(f"Year {year}:")
    print(table)
    print()

Year 2019:
+----+------------+---------+--------------+
|    | Language   |   Count |   Percentage |
|----+------------+---------+--------------|
|  0 | TypeScript |     231 |     12.1451  |
|  1 | Go         |     139 |      7.3081  |
|  2 | JavaScript |     128 |      6.72976 |
|  3 | Python     |     126 |      6.62461 |
|  4 | Java       |      96 |      5.04732 |
|  5 | Rust       |      93 |      4.88959 |
|  6 | C++        |      65 |      3.41746 |
|  7 | Ruby       |      62 |      3.25973 |
|  8 | PHP        |      55 |      2.89169 |
|  9 | C          |      46 |      2.41851 |
| 10 | C#         |      43 |      2.26078 |
| 11 | Kotlin     |      28 |      1.47213 |
| 12 | Shell      |      27 |      1.41956 |
| 13 | Swift      |      22 |      1.15668 |
| 14 | Other      |      87 |      4.57413 |
+----+------------+---------+--------------+

Year 2020:
+----+------------+---------+--------------+
|    | Language   |   Count |   Percentage |
|----+------------+---------+---

## Draw a line chart representing the evolution of the languages over the years

- This plot will represent the evolution of the use of the different languages over the years.
- The languages that are less than 1% of the total will be grouped as Other.
- The percentage of each language will be represented on the y-axis and the year on the x-axis.

In [11]:
'''
The idea is to create a line chart using Altair.
To do so it is needed to regroup the different dataframes into a single one with a structure of year,language,percentage
'''

data = []

# The loop below will create a dictionary with the year, language and percentage for each year
# And then append it to the data list in order to later create a dataframe
for year, df in resulting_repartition:
    for _, row in df.iterrows():
        data.append({
            'Year': year,
            'Language': row['Language'],
            'Percentage': row['Percentage']
        })

resulting_df = pd.DataFrame(data)
resulting_df2= resulting_df[resulting_df['Language'] != 'Other'].copy()

# Random color palette generated thanks to ai.
color_palette = [
    '#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd',
    '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf',
    '#aec7e8', '#ffbb78', '#98df8a', '#ff9896', '#c5b0d5',
    '#c49c94', '#f7b6d2', '#c7c7c7', '#dbdb8d', '#9edae5'
]

'''
This part is used to create the line chart with Altair
'''
chart = alt.Chart(resulting_df2).mark_line(point=True).encode(
    x='Year:O',
    y=alt.Y('Percentage:Q', scale=alt.Scale(type='log')),
    color=alt.Color('Language:N', scale=alt.Scale(range=color_palette)),
    tooltip=['Year', 'Language', 'Percentage']
).properties(
    width=800,
    height=700,
    title='Evolution of the language use over the years in GitHub repositories using GHA workflows',
).interactive()

chart.show()

alt.Chart(...)

## Draw a pie chart for each year

In [12]:
years = resulting_df['Year'].unique()
years.sort()
pie_charts = []
for year in years:
    pie_charts.append(pie_chart_by_year(resulting_df, year))

[chart.show() for chart in pie_charts]

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

[None, None, None, None, None, None]

## Snapshot with filter the uid history of invalid workflows

- This was another idea of filtering workflows that was based on deleting all records of each uid that has at least one invalid workflow (valid_yaml = False).
- This method is more aggressive than the previous one as it will delete all the records of a uid that has at least one invalid workflow.

In [7]:
invalid_uids = df.loc[df['valid_yaml'] == False, 'uid'].unique()
print(f"Number of invalid uids : {len(invalid_uids)}")
print(f"Number of records that corresponds to these invalids uids : {df['uid'].isin(invalid_uids).sum()}\n")

print(f"Number of records in the dataframe before : {df.shape[0]}")
print(f"Total number of uids before filtering : {df['uid'].nunique()}\n")

filtered_df2 = delete_uid_with_invalid_yaml(df)
print(f"Number of records in the dataframe once {filtered_df2.shape[0]}")
print(f"Total number of uids after filtering : {filtered_df2['uid'].nunique()}")

year = 2019
while year <= 2024:
    # Snapshot with the second method of filtering
    snapshot2 = extract_snapshot(filtered_df2, year)

    # Drop the deleted workflows
    snapshot2 = snapshot2.dropna(subset=['file_hash'])

    print(f"Year {year}:")
    print(f"Number of workflows in the snapshot: {snapshot2.shape[0]}\n")

Number of invalid uids : 8396
Number of records that corresponds to these invalids uids : 258040

Number of records in the dataframe before : 2595399
Total number of uids before filtering : 219460

Number of records in the dataframe once 2337359
Total number of uids after filtering : 211064
